In [11]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup as bsp
import matplotlib
import matplotlib.pyplot as plt
import sqlite3
import re
from datetime import datetime, timedelta,date
import sys

In [12]:
url='https://search.naver.com/search.naver?where=news&query=아모레퍼시픽&sm=tab_srt&sort=1&photo=0&field=0&reporter_article=&pd=0&ds=&de=&docid=&nso=so%3Add%2Cp%3Aall%2Ca%3Aall&mynews=1&refresh_start=0&related=0'
url

'https://search.naver.com/search.naver?where=news&query=아모레퍼시픽&sm=tab_srt&sort=1&photo=0&field=0&reporter_article=&pd=0&ds=&de=&docid=&nso=so%3Add%2Cp%3Aall%2Ca%3Aall&mynews=1&refresh_start=0&related=0'

In [13]:
header={'''
        "authority": "search.naver.com"
        "method": "GET"
        "path": "/search.naver?&where=news&query=%EC%95%84%EB%AA%A8%EB%A0%88%ED%8D%BC%EC%8B%9C%ED%94%BD&sm=tab_pge&sort=1&photo=0&field=0&reporter_article=&pd=0&ds=&de=&docid=&nso=so:dd,p:all,a:all&mynews=1&start=1&refresh_start=0"
        "scheme": "https"
        "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9"
        "accept-encoding": "gzip, deflate, br"
        "accept-language": "en-US,en;q=0.9,ko;q=0.8"
        "cache-control": "max-age=0"
        "cookie":" NNB=PCAHGC4CUI2WA; NRTK=ag#all_gr#1_ma#-2_si#0_en#0_sp#0; NDARK=Y; nx_ssl=2; ASID=3a4c821300000177fffbeb9300000066; nso_open=1; news_office_checked=1023; news_my_status=1; page_uid=haK4YlprvN8ssOxOvIhssssstpK-370930"
        "referer": "https://search.naver.com/search.naver?&where=news&query=%EC%95%84%EB%AA%A8%EB%A0%88%ED%8D%BC%EC%8B%9C%ED%94%BD&sm=tab_pge&sort=1&photo=0&field=0&reporter_article=&pd=0&ds=&de=&docid=&nso=so:dd,p:all,a:all&mynews=1&start=21&refresh_start=0"
        # "sec-ch-ua": "Google Chrome";v="89", "Chromium";v="89", ";Not A Brand";v="99"
        "sec-ch-ua-mobile": "?0"
        "sec-fetch-dest": "document"
        "sec-fetch-mode": "navigate"
        "sec-fetch-site": "same-origin"
        "sec-fetch-user": "?1"
        "upgrade-insecure-requests": "1"
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36"

'''}
       

In [14]:
res = requests.get(url,'header=header') 
src=res.content.strip()
htmls=bsp(src,'html.parser')
#htmls
tgts=htmls.select('ul.list_news > li')
#tgts

In [15]:
def calcDate(txt=''):
    #print(txt)
    rdate=''
    nowd=datetime.now()
    pattern='[^\d]'
    num=int(re.sub(pattern,'',txt))
    if('일' in txt):
        res=nowd + timedelta(days=num*-1)
    elif('.' in txt):
        ds=txt.replace('.','-')
        #print(ds)
        res=ds+' 00:00:00.000000'
    else:
        res=nowd + timedelta(hours=num*-1)
    return res

In [16]:
def makeKey(txt=''):
    txt_=re.sub('http\S*//',' ',txt)
    key=re.sub('[^a-zA-Z0-9%]','',txt_)
    return key

In [17]:
def getData(whr=''):
    dbname='amorenews.db'
    with sqlite3.connect(dbname) as conn:
        cursor=conn.cursor()
        sql='select * from anews '+whr
        res=cursor.execute(sql).fetchall()
    return(res)

In [18]:
def insMain(d):
    dbname='amorenews.db'
    with sqlite3.connect(dbname) as conn:
        cursor=conn.cursor()
        sql='select * from anews where akey="'+d[5]+'"'
        res=cursor.execute(sql).fetchall()
        if(len(res)>0):
            print("이미 존재 합니다.:"+d[1])
        else:
            sql='insert into anews (acate,atitle,acomp,adate,aurl,akey) values(?,?,?,?,?,?)'
            res=cursor.execute(sql,d)
            print('.',end='')

In [19]:
def getPage(url):
    res=requests.get(url)
    src=res.content.strip()
    htmls=bsp(src,'html.parser')
    print(url)
    tgts=htmls.select('ul.list_news > li')
    if(len(tgts)==0):
        print('데이터가 없습니다.'+url)
        sys.exit(1)
    for tgt in tgts:
        mainarti=tgt.select('div.news_wrap')
        subartis=tgt.select('ul.list_cluster > li.sub_bx')
        if(len(mainarti)>0):
            title_= mainarti[0].select('a.news_tit')[0].text
            title=re.sub('[^ㄱ-힣0-9 a-zA-Z]',' ',title_)
            comp=mainarti[0].select('.info.press')[0].text.replace('언론사 선정','')
            date_=mainarti[0].select('span.info')[0].text.replace('시간 전','')
            date=calcDate(date_)
            url=mainarti[0].select('a.news_tit')[0]['href']
            key=makeKey(url)
            opt=1
            d=(opt,title,comp,date,url,key)
            insMain(d)
            for subarti in subartis:
                title_= subarti.select('a.sub_tit')[0].text
                title=re.sub('[^ㄱ-힣0-9 a-zA-Z]',' ',title_)
                comp=subarti.select('.sub_txt.press')[0].text.replace('언론사 선정','')
                date_=subarti.select('span.sub_txt')[0].text.replace('시간 전','').replace('네이버뉴스','')
                date=calcDate(date_)
                url=subarti.select('a.sub_tit')[0]['href']
                key=makeKey(url)
                opt=2
                d=(opt,title,comp,date,url,key)
                insMain(d)


In [20]:
urlsrc='https://search.naver.com/search.naver?&where=news&query={0}&nso={1}&start={2}'
for i in range(1,101,10):
    url=urlsrc.format('아모레퍼시픽','조선일보',i)
    print(url)
    getPage(url)

https://search.naver.com/search.naver?&where=news&query=아모레퍼시픽&nso=조선일보&start=1
https://search.naver.com/search.naver?&where=news&query=아모레퍼시픽&nso=조선일보&start=1
......................https://search.naver.com/search.naver?&where=news&query=아모레퍼시픽&nso=조선일보&start=11
https://search.naver.com/search.naver?&where=news&query=아모레퍼시픽&nso=조선일보&start=11
..........https://search.naver.com/search.naver?&where=news&query=아모레퍼시픽&nso=조선일보&start=21
https://search.naver.com/search.naver?&where=news&query=아모레퍼시픽&nso=조선일보&start=21
..........https://search.naver.com/search.naver?&where=news&query=아모레퍼시픽&nso=조선일보&start=31
https://search.naver.com/search.naver?&where=news&query=아모레퍼시픽&nso=조선일보&start=31
..........https://search.naver.com/search.naver?&where=news&query=아모레퍼시픽&nso=조선일보&start=41
https://search.naver.com/search.naver?&where=news&query=아모레퍼시픽&nso=조선일보&start=41
..........https://search.naver.com/search.naver?&where=news&query=아모레퍼시픽&nso=조선일보&start=51
https://search.naver.com/search.naver?&where=news